In [1]:
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, VGAE, GatedGraphConv, GCN
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import DataLoader

from dataset_processing import RNADataset


/home/vdshk/SecondaryStructurePredictionGNN/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
dataset = RNADataset(root="./data/")
# dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)


In [14]:
print(train_data[3].x)
print(train_data[3].edge_index)
print(train_data[3].edge_weight)

tensor([[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]])
tensor([[ 0,  1,  1,  2,  1, 13,  2,  3,  2, 12,  3,  4,  3, 11,  4,  5,  4, 10,
          5,  6,  5,  9,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13],
        [ 1,  0,  2,  1, 13,  1,  3,  2, 12,  2,  4,  3, 11,  3,  5,  4, 10,  4,
          6,  5,  9,  5,  7,  6,  8,  7,  9,  8, 10,  9, 11, 10, 12, 11, 13, 12]])
tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
  

In [15]:
def del_diags(y):
    main_diag = torch.ones([1, y.size(0)])
    diag1 = torch.ones([1, y.size(0) - 1])
    return y * (torch.diag_embed(main_diag).to(device) + 
            torch.diag_embed(diag1, offset=1).to(device) + 
            torch.diag_embed(diag1, offset=-1).to(device) - 1) * (-1)

In [58]:
def precision(y_pred, y_true):
    y_pred = del_diags(y_pred)
    y_pred[(y_pred > 0.5)] = 1
    y_pred[(y_pred <= 0.5)] = 0 
    
    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):
    y_pred = del_diags(y_pred)
    y_pred[(y_pred > 0.5)] = 1
    y_pred[(y_pred <= 0.5)] = 0
    
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f_loss(y_pred, y_true):
    y_pred = del_diags(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    k = 0.001
    k1 = 1 - torch.abs(precision - recall)

#     k2 = 1 - torch.abs(K.mean(precision) - K.mean(recall))
    #calculate upgraded f1 score
    f1 = ((1 + k * k) * precision * recall) / (k * k * precision + recall + epsilon)
#     tw = K.sum(K.cast(y_true * y_pred, ’float32’), axis=[1, 2, 3])
#     fw = K.sum(K.cast((1 - y_true) * y_pred, ’float32’), axis=[1, 2, 3])
#     fb = K.sum(K.cast(y_true * (1 - y_pred), ’float32’), axis=[1, 2, 3])
    return 1 - f1


Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/vdshk/SecondaryStructurePredictionGNN/venv/lib/python3.8/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/vdshk/SecondaryStructurePredictionGNN/venv/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/disk.py", line 76, in start
    self.metrics_monitor.start()
  File "/home/vdshk/SecondaryStructurePredictionGNN/venv/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [59]:
class GatedGCNModel(torch.nn.Module):
    def __init__(self, num_layers, out_channels):
        super(GatedGCNModel, self).__init__()
        self.ggcn = GatedGraphConv(out_channels, num_layers)
#         self.gcn = GCN(out_channels, hidden_channels, 2, 64, dropout=dropout)

    def forward(self, x, edge_index, edge_weight):
        x = self.ggcn(x, edge_index)
#         x = self.gcn(x.to(torch.float), edge_index)
        prob_adj = (x @ x.t()).sigmoid()

#         prob_adj = (x @ x.t())

#         return (prob_adj > 0).nonzero(as_tuple=False).t()
        return prob_adj


In [60]:
hidden_channels = 128
out_channels = 200
num_layers = 12
num_features = dataset.num_features


dr = 0.7
lr = 0.000001
epochs = 1500
beta = 0.001


model = GatedGCNModel(num_layers, out_channels)
# model = torch.load("./models1/GatedGCN_6_4e-05_Adam_200_2_60.pt")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.99))
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
# optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# criterion = torch.nn.BCELoss()
# criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.KLDivLoss()

# def RMSELoss(y_pred, y_true):
#     return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

criterion = f_loss

cuda


In [61]:
run_name = "wGatedGCN_" + str(num_layers) + "_" + str(lr) + "_" + "Adam_" + str(out_channels) + "_" + str(beta)


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "GatedGCN",
    "epochs": epochs,
    "optimizer": "Adam",
    "out_channels": out_channels,
    "loss": "f_loss",
    "beta": beta,
    "train:val:test": "655:218:218"
    },
    name=run_name
)

epsilon = 1e-10

train_f1,▁▁▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
train_loss,██▅▅▅▄▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁
train_precision,▁▁▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
train_recall,█▂▄▄▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_f1,▁▂▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇████████████▇██████████
val_loss,█▇▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
val_precision,▁▂▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇████████████▇██████████
val_recall,█▁▅▄▅▄▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_f1,0.09415
train_loss,0.91211
train_precision,0.05019


In [62]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
        for g in tqdm(train_data, ncols=100):
            g.to(device)
            optimizer.zero_grad()

            out = model(g.x, g.edge_index, g.edge_weight)
            y_true = g.adj_mat
            loss = criterion(out, y_true.to(torch.float32))            
            
            loss.backward()
            optimizer.step()
            
#             out = out.sigmoid()
            
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
            for g in tqdm(val_data, ncols=100):
                g.to(device)
                out = model(g.x, g.edge_index, g.edge_weight)
        
                y_true = g.adj_mat
                loss = criterion(out, y_true.to(torch.float32))
                                
                val_loss.append(loss.item())
                
#                 out = out.sigmoid()
                
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [63]:
train()
torch.save(model, "./models1/" + run_name + "_" + str(epochs) + ".pt")

100%|█████████████████████████████████████████████████████████████| 655/655 [00:07<00:00, 90.11it/s]


Epoch: 001, loss: 0.979914026769973, f1: 0.03872048743870516, precision: 0.019748928514131493, recall: 0.983686715728454


100%|████████████████████████████████████████████████████████████| 218/218 [00:00<00:00, 252.74it/s]


val_loss: 0.9820497454306402, val_f1: 0.03444058747447869, val_precision: 0.017528408235836124, val_recall: 0.9796499026888007


100%|█████████████████████████████████████████████████████████████| 655/655 [00:07<00:00, 89.90it/s]


...

val_loss: 0.9645191622436593, val_f1: 0.06800238413222574, val_precision: 0.03560176636669101, val_recall: 0.7562925980315296


100%|█████████████████████████████████████████████████████████████| 655/655 [00:07<00:00, 92.02it/s]


Epoch: 1500, loss: 0.9566904067993164, f1: 0.08254000248914298, precision: 0.043823807314038274, recall: 0.7082012232705837


100%|████████████████████████████████████████████████████████████| 218/218 [00:00<00:00, 273.15it/s]

val_loss: 0.9609704326599016, val_f1: 0.07491398869919151, val_precision: 0.03950056667683151, val_recall: 0.7240127902990634


In [ ]:
run_name = "GatedGCN_" + str(num_layers) + "_" + "140" + "_" + str(lr) + "_" + "Adam_" + str(out_channels)


torch.save(model, "./models/" + run_name + ".pt")